In [6]:
import time
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

prompts = [
    "Write a short story about a robot discovering human emotions.",
    "Compose a poem about the changing seasons.",
    "Describe a futuristic city in the year 2150.",
    "Write a dialogue between an alien and a human discussing the meaning of life.",
    "Write a haiku about the ocean at night.",
    
    "Write a Python function to calculate the Fibonacci sequence.",
    "Explain the concept of recursion with an example in Python.",
    "Write a SQL query to select the top 5 highest-paid employees from a database.",
    "Generate a Python class to represent a deck of cards and a function to shuffle it.",
    "Write a regex pattern to extract email addresses from a text.",
    
    "Solve the equation: x^2 + 3x - 4 = 0 and explain the solution.",
    "Find the derivative of f(x) = x^3 - 2x + 1.",
    "Calculate the area under the curve y = x^2 from x = 0 to x = 2.",
    "Solve this matrix multiplication problem: [[2, 3], [1, 4]] * [[1, 2], [0, 1]].",
    "Find the greatest common divisor of 456 and 123 using Euclid’s algorithm.",
    
    "A train travels from city A to city B at 60 mph and returns at 40 mph. What is the average speed for the whole trip?",
    "If it takes 3 people 4 hours to paint a house, how long will it take 5 people working at the same rate?",
    "A farmer has 17 cows, and all but 8 run away. How many are left?",
    "If two cats catch two mice in two minutes, how many cats are needed to catch 100 mice in 100 minutes?",
    "A man buys a watch for $40, sells it for $50, then buys it back for $60 and sells it again for $70. What is his total profit?",
    "write a poem about sun and moon"
]

total_time = 0
total_tokens = 0
time_to_first_token_list = []
throughput_list = []
generated_results = []

for prompt in prompts:
    start_time = time.time()
    first_token_time = None
    
    stream = False
    completion = client.completions.create(
        model="bigscience/bloomz-1b1",
        prompt=prompt,
        echo=False,
        n=1,
        stream=stream,
    )
    
    if not stream:
        first_token_time = time.time() - start_time
        total_tokens += completion.usage.completion_tokens
    
    completion_time = time.time() - start_time
    total_time += completion_time
    
    if completion_time > 0:
        throughput = completion.usage.completion_tokens / completion_time
    else:
        throughput = 0
    
    time_to_first_token_list.append(first_token_time)
    throughput_list.append(throughput)
    
    generated_results.append({
        "Prompt": prompt,
        "Completion": completion.choices[0].text.strip(),
        "Time to First Token (s)": first_token_time,
        "Total Time (s)": completion_time,
        "Throughput (tokens/s)": throughput,
        "Tokens Generated": completion.usage.completion_tokens,
    })

average_time = total_time / len(prompts)
average_time_to_first_token = sum(time_to_first_token_list) / len(time_to_first_token_list)
average_throughput = sum(throughput_list) / len(throughput_list)

results = {
    "Total Prompts": len(prompts),
    "Total Time (s)": total_time,
    "Average Time per Prompt (s)": average_time,
    "Average Time to First Token (s)": average_time_to_first_token,
    "Average Throughput (tokens/s)": average_throughput,
    "Total Tokens Generated": total_tokens,
}

print("\n=== Model Performance Report ===")
for key, value in results.items():
    print(f"{key}: {value}")

print("\n=== Generated Results ===")
for i, result in enumerate(generated_results):
    print(f"\nPrompt {i + 1}: {result['Prompt']}")
    print(f"Completion: {result['Completion']}")
    print(f"Time to First Token (s): {result['Time to First Token (s)']:.4f}")
    print(f"Total Time (s): {result['Total Time (s)']:.4f}")
    print(f"Throughput (tokens/s): {result['Throughput (tokens/s)']:.4f}")
    print(f"Tokens Generated: {result['Tokens Generated']}")
    print("-" * 60)



=== Model Performance Report ===
Total Prompts: 21
Total Time (s): 0.907310962677002
Average Time per Prompt (s): 0.04320528393700009
Average Time to First Token (s): 0.043203172229585196
Average Throughput (tokens/s): 133.00641918069965
Total Tokens Generated: 146

=== Generated Results ===

Prompt 1: Write a short story about a robot discovering human emotions.
Completion: For some reason, while its humans love handling things, leading hurts, dealing
Time to First Token (s): 0.1289
Total Time (s): 0.1289
Throughput (tokens/s): 124.1178
Tokens Generated: 16
------------------------------------------------------------

Prompt 2: Compose a poem about the changing seasons.
Completion: 
Time to First Token (s): 0.0222
Total Time (s): 0.0222
Throughput (tokens/s): 45.0124
Tokens Generated: 1
------------------------------------------------------------

Prompt 3: Describe a futuristic city in the year 2150.
Completion: As Pluto enters the last orbit of the Sun, Atlantis's great rivals, the

In [7]:
import time
from openai import OpenAI
from multiprocessing import Pool, cpu_count

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

def process_prompt(prompt):
    start_time = time.time()

    completion = client.completions.create(
        model="bigscience/bloomz-1b1",
        prompt=prompt,
        echo=False,
        n=1,
        stream=False,
    )

    first_token_time = time.time() - start_time
    completion_time = time.time() - start_time
    tokens_generated = completion.usage.completion_tokens
    
    throughput = tokens_generated / completion_time if completion_time > 0 else 0
    
    result = {
        "Prompt": prompt,
        "Completion": completion.choices[0].text.strip(),
        "Time to First Token (s)": first_token_time,
        "Total Time (s)": completion_time,
        "Throughput (tokens/s)": throughput,
        "Tokens Generated": tokens_generated,
    }

    return result


In [8]:
num_workers = 1
print(f"Using {num_workers} parallel workers")

with Pool(processes=num_workers) as pool:
    results = pool.map(process_prompt, prompts)

total_time = sum(r["Total Time (s)"] for r in results)
total_tokens = sum(r["Tokens Generated"] for r in results)
time_to_first_token_list = [r["Time to First Token (s)"] for r in results]
throughput_list = [r["Throughput (tokens/s)"] for r in results]

average_time = total_time / len(prompts)
average_time_to_first_token = sum(time_to_first_token_list) / len(time_to_first_token_list)
average_throughput = sum(throughput_list) / len(throughput_list)

summary = {
    "Total Prompts": len(prompts),
    "Total Time (s)": total_time,
    "Average Time per Prompt (s)": average_time,
    "Average Time to First Token (s)": average_time_to_first_token,
    "Average Throughput (tokens/s)": average_throughput,
    "Total Tokens Generated": total_tokens,
}

print("\n=== Model Performance Report ===")
for key, value in summary.items():
    print(f"{key}: {value}")

print("\n=== Generated Results ===")
for i, result in enumerate(results):
    print(f"\nPrompt {i + 1}: {result['Prompt']}")
    print(f"Completion: {result['Completion']}")
    print(f"Time to First Token (s): {result['Time to First Token (s)']:.4f}")
    print(f"Total Time (s): {result['Total Time (s)']:.4f}")
    print(f"Throughput (tokens/s): {result['Throughput (tokens/s)']:.4f}")
    print(f"Tokens Generated: {result['Tokens Generated']}")
    print("-" * 60)

Using 1 parallel workers

=== Model Performance Report ===
Total Prompts: 21
Total Time (s): 0.9360940456390381
Average Time per Prompt (s): 0.04457590693519229
Average Time to First Token (s): 0.0445756231035505
Average Throughput (tokens/s): 156.33030239674972
Total Tokens Generated: 166

=== Generated Results ===

Prompt 1: Write a short story about a robot discovering human emotions.
Completion: A bot will use a practice head as it talks to humans. It was once
Time to First Token (s): 0.0911
Total Time (s): 0.0911
Throughput (tokens/s): 175.6151
Tokens Generated: 16
------------------------------------------------------------

Prompt 2: Compose a poem about the changing seasons.
Completion: Winter the desert -- so beautiful, which overlooks the sea -- the different colors
Time to First Token (s): 0.0790
Total Time (s): 0.0790
Throughput (tokens/s): 202.6503
Tokens Generated: 16
------------------------------------------------------------

Prompt 3: Describe a futuristic city in the